# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [97]:
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [98]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/Users/george/Documents/Elite Dangerous/Data engineering/Udacity/Data/Data modeing Cassandra project/workspace/home
['/Users/george/Documents/Elite Dangerous/Data engineering/Udacity/Data/Data modeing Cassandra project/workspace/home/event_data/2018-11-15-events.csv', '/Users/george/Documents/Elite Dangerous/Data engineering/Udacity/Data/Data modeing Cassandra project/workspace/home/event_data/2018-11-22-events.csv', '/Users/george/Documents/Elite Dangerous/Data engineering/Udacity/Data/Data modeing Cassandra project/workspace/home/event_data/2018-11-09-events.csv', '/Users/george/Documents/Elite Dangerous/Data engineering/Udacity/Data/Data modeing Cassandra project/workspace/home/event_data/2018-11-18-events.csv', '/Users/george/Documents/Elite Dangerous/Data engineering/Udacity/Data/Data modeing Cassandra project/workspace/home/event_data/2018-11-04-events.csv', '/Users/george/Documents/Elite Dangerous/Data engineering/Udacity/Data/Data modeing Cassandra project/workspace/home/event_

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [99]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [100]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

    print(line)

6821
['Deas Vail', 'Logged In', 'Elijah', 'M', '0', 'Davis', '237.68771', 'free', 'Detroit-Warren-Dearborn, MI', 'PUT', 'NextSong', '1.54077E+12', '985', 'Anything You Say (Unreleased Version)', '200', '1.54361E+12', '5']


## The event_datafile_new.csv contains the following columns:
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [101]:
# This should make a connection to a Cassandra instance on local
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [102]:
session

#### Create Keyspace

In [103]:
create_keyspace_query = "CREATE KEYSPACE IF NOT EXISTS sparkify \
                        WITH REPLICATION = {'class': 'SimpleStrategy', \
                                            'replication_factor':1}"
session.execute(create_keyspace_query)

#### Set Keyspace

In [104]:
session.set_keyspace('sparkify')

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [105]:
# column indexes
artist = 0
firstName = 1
gender = 2
itemInSession=3
lastName=4
length=5
level=6
location=7
sessionId=8
song=9
userId=10

## Query 1
### Table Name: songs_per_sessions
**Purpose:**
- Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

- To retrieve the artist, song title and length based on sessionId and itemInSession.

**Primary Key:**
- **Partition Key:**
SessionId and  (Used to partition data by SessionId)

- **Clustering Key:**
itemInSession (Used to sort the songs within a session by the order itemInSession.)

- **Reasoning:** Sessions are unique per user, but repeats as many times as itemInSessions, the combination of both are unique, sorting by itemInSession makes sense

In [106]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = 'CREATE TABLE songs_per_sessions (\
        sessionId int,\
        itemInSession int, \
        artist text,\
        song text,\
        length decimal, \
        PRIMARY KEY(sessionId,itemInSession) )'
session.execute(query)
                    

In [107]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_per_sessions (sessionId,itemInSession,artist,song,length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[sessionId]),int(line[itemInSession]),line[artist],line[song], float(line[length])))

#### Do a SELECT to verify that the data have been inserted into each table

In [108]:
select_songs_per_sessions_query = 'SELECT artist,song,length FROM songs_per_sessions \
                                   WHERE sessionId=338 and itemInSession = 4'
rows = session.execute(select_songs_per_sessions_query)
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=Decimal('495.3073'))


## Query 2
### Table Name: songs_per_user
**Purpose:**
- Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

- To retrieve the artist, song title sorted by itemInSession and user's first and last name for a specific userId and sessionId.

**Primary Key:**
- **Partition Key:**
userId and sessionId

- **Clustering Key:**
itemInSession (Used to sort the songs within a session by the order itemInSession.)

- **Reasoning:** Query will filter by userId and sessionId so we need those as a composed primary key for performance, if we want to sort by itemInSession we need to use it as a clusteringId

In [109]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = 'CREATE TABLE songs_per_user (\
        userid int, \
        sessionId int,\
        itemInSession int, \
        artist text,\
        song text,\
        firstName text,\
        lastName text,\
        PRIMARY KEY((userid,sessionId),itemInSession))'
session.execute(query)

In [110]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_per_user (userid,sessionId,itemInSession,artist,song,firstName,lastName)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[userId]),int(line[sessionId]),int(line[itemInSession]),line[artist],line[song],line[firstName],line[lastName]))

In [111]:
select_songs_per_sessions_query = 'SELECT artist,song,itemInSession,firstName,lastName FROM songs_per_user \
                                   WHERE userid = 10 and sessionid = 182'
rows = session.execute(select_songs_per_sessions_query)
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", iteminsession=0, firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', iteminsession=1, firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', iteminsession=2, firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', iteminsession=3, firstname='Sylvie', lastname='Cruz')


## Query 3
### Table Name: user_names_per_song
**Purpose:**
- Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

- To retrieve the user first and last names for all users who listened to a specific song

**Primary Key:**
- **Partition Key:**
song

- **Clustering Key:**
userId

- **Reasoning:** song primary key ensures all users who listened to the same song are stored together making the filter efficient, adding userId as the clustering key guarantees uniqueness per user.

In [112]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = 'CREATE TABLE user_names_per_song (\
        song text,\
        userid int,\
        firstName text,\
        lastName text,\
        PRIMARY KEY(song,userid)\
        )'
session.execute(query)
                    

In [113]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_names_per_song (song,userid,firstName,lastName)"
        query = query + "VALUES (%s,%s,%s,%s)"
        session.execute(query, (line[song],int(line[userId]),line[firstName],line[lastName]))

In [114]:
select_songs_per_sessions_query = "SELECT firstName,lastName FROM user_names_per_song \
                                   WHERE song ='All Hands Against His Own'"
rows = session.execute(select_songs_per_sessions_query)
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [115]:
drop_songs_per_session_query = 'DROP TABLE songs_per_sessions'
session.execute(drop_songs_per_session_query)

drop_songs_per_user_query = 'DROP TABLE songs_per_user'
session.execute(drop_songs_per_user_query)

drop_songs_per_user_query = 'DROP TABLE user_names_per_song'
session.execute(drop_songs_per_user_query)

### Close the session and cluster connection¶

In [116]:
session.shutdown()
cluster.shutdown()